In [1]:
import mlflow
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
df = pd.read_csv('IMDB.csv')
# df = df.sample(500)
# df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
0,Film version of Sandra Bernhard's one-woman of...,negative
1,I switched this on (from cable) on a whim and ...,positive
2,The `plot' of this film contains a few holes y...,negative
3,"Some amusing humor, some that falls flat, some...",negative
4,What can you say about this movie? It was not ...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
0,film version sandra bernhard s one woman off b...,negative
1,switched from cable whim treated quite surpris...,positive
2,plot film contains hole could drive massive tr...,negative
3,amusing humor fall flat decent acting quite at...,negative
4,say movie terrible good two day earlier watche...,negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    517
positive    483
Name: count, dtype: int64

In [ ]:
# Discarding sentiments other than positive and negative
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
0,film version sandra bernhard s one woman off b...,0
1,switched from cable whim treated quite surpris...,1
2,plot film contains hole could drive massive tr...,0
3,amusing humor fall flat decent acting quite at...,0
4,say movie terrible good two day earlier watche...,0


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/dhruvchandralohani/Cloud-Native-MLOps-Platform-for-Text-Classification.mlflow')
dagshub.init(repo_owner='dhruvchandralohani', repo_name='Cloud-Native-MLOps-Platform-for-Text-Classification', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=1b9b83d7-634c-4397-acc3-48aa20f2d13a&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=5284624db082cc67f141dcacd0ecaaeecda7d29427522c01638bc24dd1175aef




Accessing as dhruvchandralohani

Initialized MLflow to track repo "dhruvchandralohani/Cloud-Native-MLOps-Platform-for-Text-Classification"

Repository dhruvchandralohani/Cloud-Native-MLOps-Platform-for-Text-Classification initialized!

<Experiment: artifact_location='mlflow-artifacts:/b9e1061eafd14ef8a8c49dc791b71923', creation_time=1763721787026, experiment_id='0', last_update_time=1763721787026, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [12]:
import mlflow
import logging
import time
import tempfile, shutil
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 1000)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        # DAGsHub's MLflow endpoint doesn't support the CreateLoggedModel endpoint used
        # by mlflow.sklearn.log_model. Save locally and upload artifacts instead.
        tmp_dir = tempfile.mkdtemp(prefix="sklearn_model_")
        try:
            mlflow.sklearn.save_model(model, tmp_dir)
            mlflow.log_artifacts(tmp_dir, artifact_path="model")
        finally:
            shutil.rmtree(tmp_dir, ignore_errors=True)

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")
        mlflow.log_metric("execution_time_seconds", end_time - start_time)

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2026-02-08 18:44:01,776 - INFO - Starting MLflow run...
2026-02-08 18:44:03,844 - INFO - Logging preprocessing parameters...
2026-02-08 18:44:04,839 - INFO - Initializing Logistic Regression model...
2026-02-08 18:44:04,849 - INFO - Fitting the model...
2026-02-08 18:44:05,086 - INFO - Model training complete.
2026-02-08 18:44:05,087 - INFO - Logging model parameters...
2026-02-08 18:44:05,430 - INFO - Making predictions...
2026-02-08 18:44:05,437 - INFO - Calculating evaluation metrics...
2026-02-08 18:44:05,473 - INFO - Logging evaluation metrics...
2026-02-08 18:44:06,882 - INFO - Saving and logging the model...
2026-02-08 18:44:44,755 - INFO - Model training and logging completed in 40.91 seconds.
2026-02-08 18:44:45,070 - INFO - Accuracy: 0.74
2026-02-08 18:44:45,071 - INFO - Precision: 0.7340425531914894
2026-02-08 18:44:45,072 - INFO - Recall: 0.71875
2026-02-08 18:44:45,072 - INFO - F1 Score: 0.7263157894736842


🏃 View run stately-steed-860 at: https://dagshub.com/dhruvchandralohani/Cloud-Native-MLOps-Platform-for-Text-Classification.mlflow/#/experiments/0/runs/52d6aeb2504f4d10a55488b04772c22f
🧪 View experiment at: https://dagshub.com/dhruvchandralohani/Cloud-Native-MLOps-Platform-for-Text-Classification.mlflow/#/experiments/0
